In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import matplotlib.pyplot as plt
from pprint import pprint

# Import API key
from config import gkey

In [2]:
cities_df = pd.read_csv('Other_Data/Heatmap_Data.csv')
del cities_df["Unnamed: 0"]
del cities_df["Population"]
del cities_df["Poverty Count"]

In [4]:
cities_df["State"] = "Iowa"
cities_df['lat'] = ""
cities_df['lng'] = ""
cities_df = cities_df[["City", "State", "Zip Code", "Total Poverty Rate (%)", "lat", "lng"]]
cities_df.head()
cities_df.to_csv("Cities Poverty.csv")

In [4]:
params = {"key":gkey}
for index, row in cities_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    city = row['City']
    state = row['State']
    params['address'] = f"{city},{state}"
    cities_lat_lng = requests.get(base_url, params=params).json()
    cities_df.loc[index, "lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    cities_df.loc[index, "lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

In [5]:
cities_df.head()

,City,State,Zip Code,Total Poverty Rate (%),lat,lng
0,Ackworth,Iowa,50001,19.034853,41.3669,-93.4727
1,Adair,Iowa,50002,11.157311,41.5004,-94.6434
2,Adel,Iowa,50003,6.964964,41.6144,-94.0175
3,Albion,Iowa,50005,1.871921,42.1124,-92.99
4,Alden,Iowa,50006,9.500308,42.5203,-93.376


In [6]:
gmaps.configure(api_key=gkey)

In [13]:
locations = cities_df[["lat", "lng"]].astype(float)

poverty_rate = cities_df["Total Poverty Rate (%)"].astype(float)

In [14]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
heat_layer.dissipating = False
heat_layer.max_intensity = 75
heat_layer.point_radius = 0.3

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [9]:
stores_df = pd.read_csv("liquor_store_ia.csv")

del stores_df["Unnamed: 0"]
del stores_df["Status"]
del stores_df["Zip Code"]
stores_df.head()

,Address,City,State,lat,lng
0,200 State Po Box 98,Guthrie Center,IA,41.677408,-94.502649
1,1111 8TH ST,Boone,IA,42.064750,-93.876110
2,3140 Agency,Burlington,IA,40.816170,-91.136651
3,939 Angular,Burlington,IA,40.801319,-91.110647
4,"1843 Johnson Avenue, N.W.",Cedar Rapids,IA,41.973912,-91.698396


In [15]:
locations = stores_df[["lat", "lng"]].astype(float)

store_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,)
    
fig = gmaps.figure()

fig.add_layer(store_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(store_layer)

fig

Figure(layout=FigureLayout(height='420px'))